In [1]:
import json
import numpy as np
import os

from copy import deepcopy
from itertools import product

# Section 1

For Section I of this notebook, we will learn how to represent and manipulate probability mass functions with code. This will allow us to numerically check if a joint probability distribution factorises according to a specific graph.

**Useful jupyter shortcuts**: 
- when you have a cell selected, shift+enter runs that cell
- when you place your cursor inside the brackets of a function call, shift+tab shows the input parameters required and the first line of the docstring. If you continue to hold shift, and press tab again, you can see scroll through the entire docstring.
- More useful shortcuts can be found by pressing 'Help' in the toolbar and then 'Keyboard Shortcuts'

## How to represent discrete probability distributions with arrays: A 2D example

Suppose we have two discrete random variables $x_0, x_1 \in \{0, 1, 2\}$. We can represent a joint distribution $p(x_0, x_1)$ over these two variables using a *matrix*. For instance, we could have the following Joint Probability Distribution (JPD):



In [2]:
jpd = np.array([
    [0.1, 0.0, 0.0],
    [0.2, 0.35, 0.2],
    [0.1, 0.0, 0.05]
         ])

print(jpd)

[[0.1  0.   0.  ]
 [0.2  0.35 0.2 ]
 [0.1  0.   0.05]]


where the rows index states of $x_0$ and the columns index states of $x_1$. Note that this a valid distribution since all the entries are non-negative and the sum over all elements equals 1. It's good coding practice to confirm such properties:

In [3]:
def check_valid_distribution(dist):
    assert np.all(dist) >= 0, "Not all elements are non-negative"
    assert np.allclose(np.sum(dist), 1), "Sum of all probabilities equals {}, not 1".format(np.sum(jpd))
    print("Distribution is valid!")

check_valid_distribution(jpd)

Distribution is valid!


Similarly, we can represent Conditional Probability Distributions (CPDs) over $x_0, x_1$ using matrices. 
The CPD $p(x_0 | x_1)$ that we get from the above JPD can be represented as:

In [4]:
normalising_constants = jpd.sum(axis=0)
cpd = jpd/normalising_constants
print(cpd)

[[0.25 0.   0.  ]
 [0.5  1.   0.8 ]
 [0.25 0.   0.2 ]]


Note that each *column* now represents a probability distribution over $x_0$. Hence, each column sums to 1. Let's check this:

In [5]:
num_columns = cpd.shape[1]
for j in range(num_columns):
    check_valid_distribution(cpd[:, j])

Distribution is valid!
Distribution is valid!
Distribution is valid!


## Scaling up to N dimensions

Suppose now that we have $n$ variables $X = \{x_0, \ldots x_{n-1}\}$ and that we wish to represent a distribution $p(Q | E)$, where $Q, E \subseteq X$ and $Q \cap E = \emptyset$.

To explain the notation: we use '$Q$' for 'query' and '$E$' for 'evidence'. Intuitively, after observing some evidence $E$, we wish to query (i.e compute a distribution over) other variables of interest $Q$.

$Q$ and $E$ can be represented as python sets containing the indices of their respective variables. If we suppose $|Q \cup E| = k \leq n$, then the distribution $p(Q | E)$ can be represented as a $k$-dimensional array $P$, where each variable in $Q \cup E$ is associated with a dimension of P.

Of course, this requires us to specify a mapping between indices of variables in $Q \cup E$ and dimensions of $P$. We will assume an *ordering-preserving mapping*, so that the lowest index in $Q \cup E$ corresponds to the first dimension of $P$, and the highest index to the last dimension.

Thus, to represent a distribution and its domain, we use triples $(Q, E, P)$, where $E$ is allowed to be the empty set.

For example, suppose we have 8 independent categorical variables (with different-sized domains), each of which is uniformly distributed:

In [7]:
# X = {0, 1, 2, 3, 4, 5, 6, 7}  - each number represents a random variable
# Q = {2, 4, 5}
# E = {3, 7}
# P = np.ones((15, 20, 10, 8, 25)) / (15*10*8)
# dist = (Q, E, P)
# vars_to_dim_dict = {"2": 0, "3": 1, "4": 2, "5": 3, "7": 4}

where the dictionary $\texttt{vars_to_dim_dict}$ shows the correspondence between the random variables $x_i \ \in Q \cup E$ and their associated dimension in $P$.

Note that the numbers $15, 20, 10, 8, 25$ have no special significance; they are just the sizes of the state spaces for each random variable.

#### IMPORTANT:
- ***Throughtout the rest of the notebook, the term 'distribution', when used in the code, refers to $(Q, E, P)$ triplets.***
- ***Unless instructed otherwise, do not assume that $Q \cup E = \{0, \ldots, n-1\}$ (see above example).***

# QUESTION 1

Write a function $\texttt{marginalise}$ that takes as input:

- A distribution  $(Q, E, P)$.
- A set of variables to marginalise $M$.

and outputs the marginal distribution.

We've provided a helper method $\texttt{get_dims_for_vars}$ that, given a set of variables $M$, returns the corresponding dimensions in $P$.

For all questions, make sure to check that the inputs to your function are valid! We've provided a method $\texttt{is_valid_dist}$ to help you, but you will need to write your own code too. You may find python Set methods helpful here.

Do not allow your function to return 'null' distributions i.e distributions for which $Q = \emptyset$.

Finally, please write your solution to this question **without** for-loops.

In [8]:
def get_vars_to_dim_dict(Q, E):
    """Returns a dictionary mapping each variable in QuE to a dimension of P

    Args:
        Q (set): all query variables
        E (set): all evidence variables

    Returns:
        vars (dict): mapping from variables (str) to a dimension of P (int)
    """
    
    variables = sorted(Q.union(E))
    vars_to_dim_dict = {str(v): i for i, v in enumerate(variables)}
    
    return vars_to_dim_dict


def get_dims_for_vars(M, Q, E):
    """For each var in M, return its corresponding dimension in P

    Args:
        M (set): variables for which we want the corresponding dimensions
        Q (set): all query variables
        E (set): all evidence variables

    Returns:
        dims (list of ints): dimensions of P indexed by variables in M (in sorted order)
    """
    
    v_to_d_dict = get_vars_to_dim_dict(Q, E)
    dims = [v_to_d_dict[str(m)] for m in sorted(M)]
    
    return dims


In [9]:
def is_valid_dist(dist):
    """Checks if distribution is valid - raises error if not

    Args:
        dist (tuple): (Q, E, P) as defined in main text

    Returns:
        None
    """
    Q, E, P = dist
    
    assert Q not in [{}, set()], "Q equals the empty set."
    
    n_vars = len(Q.union(E))
    n_dims = len(P.shape)
    
    assert n_vars == n_dims, \
        "|Q U E| = {} is not equal to the number of dims in P, which is {}".format(n_vars, n_dims)
    
    assert Q.intersection(E) == set(), "The intersection of Q & E is non-empty. " \
        "Intersection is {}".format(Q.intersection(E))
    
    assert np.all(P >= 0), "Not all probabilities are non-negative: {}".format(P)
    
    Q_dims = get_dims_for_vars(Q, Q, E)
    sums = np.sum(P, axis=tuple(Q_dims))
                  
    assert np.allclose(sums, 1), "Distribution does not sum to 1 for all settings " \
        "of the evidence variables. Sums are: {}".format(sums)


In [10]:
def marginalise(dist, M):
    """Eliminates variables from a probability distribution via marginalisation

    Args:
        dist (tuple): (Q, E, P) as defined in main text
        M (set): indices of variables to eliminate

    Returns:
        dist (tuple): (Q\M, E, P_new)
    
    -------------------------------
    
    EXAMPLE: 
    
    Q = {0, 4, 5, 10}
    E = {3, 7}
    P = np.arange(5*2*7*4*8*6).reshape(5, 2, 7, 4, 8, 6)
    
    input_dist = (Q, E, P)
    M = {0, 5}
    
    new_Q, new_E, new_P = marginalise(input_dist, M)
    
    print(new_Q)
    >> {4, 10}
    
    print(new_E)
    >> {3, 7}
    
    print(new_P.shape)
    >> (2, 7, 8, 6)

    """
    is_valid_dist(dist)
    Q, E, P = dist
    P = deepcopy(P)  # prevents us from altering the original array
    
    # *** YOUR CODE HERE ***

# Tests

Run the cells below to check your $\texttt{marginalise}$ implementation against some test-cases

In [ ]:
def does_match_ground_truth(l, fn, mode):
    s = lambda x: set(list(x))
    a = lambda x: np.array(list(x))
    
    dist = (s(l["Q"]), s(l["E"]), l["P"])
    Q, E, P = fn(dist, s(l[mode]))
    
    does_match = True
    try:
        check(a(Q), l["Q_true"], "Q")
        check(a(E), l["E_true"], "E")
        check(P, l["P_true"], "P")
        print("**** Passed check *****")
        
    except AssertionError as e:
        print("!!!! Check failed !!!! \n \n The inputs we passed to your function were:" \
              "\n Q: {} \n E: {} \n P: {} \n {}: {} \n".format(s(l["Q"]), s(l["E"]), l["P"], mode, s(l[mode])))
              
        print("The outputs that your function returned were: " \
              "\n Q: {} \n E: {} \n P: {} \n".format(Q, E, P))
            
        print("But the actual answer is:" \
              "\n Q: {} \n E: {} \n P: {}".format(s(l["Q_true"]), s(l["E_true"]), l["P_true"]))
        does_match = False
    
    return does_match
    
def check(x, true_x, name):
    assert np.allclose(x, true_x), "Incorrect {} returned. Should be {}, but got {}".format(name, x, true_x)

In [ ]:
P1 = np.load("jpd_1.npz")["P"]

try:
    dist1 = ({i for i in range(P1.ndim)}, set(), P1)
    marginalise(dist1, {0, 1, 2})
    print("Failed check: your marginalise function should throw an error if M == Q")
except:
    print("**** Passed check *****")

In [ ]:
for i in range(1, 5):
    l = np.load("marginal_1_{}.npz".format(i))
    if not does_match_ground_truth(l, marginalise, "M"):
        break

In [ ]:
for i in range(1, 3):
    l = np.load("marginal_2_{}.npz".format(i))
    does_match_ground_truth(l, marginalise, "M")

# QUESTION 2

Write a function $\texttt{condition}$ that takes as input:

- A distribution $(Q, E, P)$.
- A set of variables to condition on, $C$.

and outputs the new conditional distribution.

It is fine to use a for-loop in your answer, but if you can vectorise the operations then that's even more efficient (and you might even find it easier).

If using for-loops, then the functions $\texttt{product}$ (which we've already imported) and $\texttt{get_multidim_slice}$ (which we implement below) may be of use to you.

If using vector operations, then $\texttt{np.transpose}$ should help you leverage numpy's inbuilt broadcasting (https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)

Again, do not allow your function to return 'null' distributions i.e distributions for which $Q = \emptyset$.

In [11]:
def get_multidim_slice(array, dim_to_index_dict):
    """Returns a list that can be used to slice into n-dim 'array' - see Example below.
    
    Args:
        array (np.ndarray): n dimensional array to slice into.
        dim_to_index_dict (dict): A key in the dict refers to a dimension of 'array';
                                  the corresponding value refers to the index at which we slice that dimension.
                                  If a dimension is not included in this dict, we keep the entirety of it.

    Returns
        slice_idxs (list of ints / slices): n-length list that can be used to slice into 'array'
    
    -------------------------------
    
    EXAMPLE: 
    
    test_array = np.arange(1*2*3*4*5*6).reshape(1, 2, 3, 4, 5, 6)
    dim_to_index_dict = {0: 0, 3: 1, 5: 2}
    
    slice_idxs = get_multidim_slice(test_array, dim_to_index_dict)
    sliced_array = test_array[slice_idxs]
    
    print(sliced_array.shape)
    >> (2, 3, 5)
    
    np.all(sliced_array == test_array[0, :, :, 1, :, 2])
    >> True
    """
    _validate_multidim_slice_input(array, dim_to_index_dict)
    
    ix = [dim_to_index_dict.get(dim, slice(None)) for dim in range(array.ndim)]
    
    return tuple(ix)


def _validate_multidim_slice_input(array, dim_to_index_dict):
    
    dims_to_slice = set(dim_to_index_dict.keys())
    all_dims = set(range(array.ndim))
    extra_dims = dims_to_slice.difference(all_dims)
    
    assert extra_dims == set(), "'dim_to_index_dict' contains keys which" \
        " are not valid dimensions of 'array', namely: {}".format(extra_dims)


In [12]:
def condition(dist, C):
    """Computes a conditional distribution

    Args:
        dist (tuple): (Q, E, P) as defined in main text
        C (set): indices of new variables to condition on (i.e add to E)

    Returns
        dist (tuple): (Q\C, EuC, P_new)
    
    -------------------------------
    
    EXAMPLE: 
    
    Q = {0, 4, 5, 10}
    E = {3, 7}
    P = np.arange(5*2*7*4*8*6).reshape(5, 2, 7, 4, 8, 6)
    
    input_dist = (Q, E, P)
    C = {0, 5}
    
    new_Q, new_E, new_P = condition(input_dist, C)
    
    print(new_Q)
    >> {4, 10}
    
    print(new_E)
    >> {0, 3, 5, 7}
    
    print(new_P.shape)
    >> (5, 2, 7, 4, 8, 6)
    
    """
    is_valid_dist(dist)
    Q, E, P = dist
    P = deepcopy(P)  # prevents us from altering the original array
    
    # *** YOUR CODE HERE ***

# Tests

Run the cells below to check your $\texttt{condition}$ implementation against some test-cases

In [ ]:
jpd1 = np.load("jpd_1.npz")["P"]
dist1 = ({i for i in range(jpd1.ndim)}, set(), jpd1)
try:
    condition(dist1, {0, 1, 2})
    print("Failed check: your condition function should throw an error if C == Q")
except:
    print("Passed check!")

In [ ]:
for i in range(1, 5):
    l = np.load("conditional_1_{}.npz".format(i))
    does_match_ground_truth(l, condition, "C")

In [ ]:
for i in range(1, 3):
    l = np.load("conditional_2_{}.npz".format(i))
    does_match_ground_truth(l, condition, "C")

In [ ]:
for i in range(1, 3):
    l = np.load("conditional_3_{}.npz".format(i))
    does_match_ground_truth(l, condition, "C")

# QUESTION 3A

Write a function $\texttt{check_equality_of_conditionals}$ that checks if $p(Q|E_1) = p(Q|E_2)$, where $E_1 \subseteq E_2$.

It is fine to use a for-loop in your answer, or you can vectorise it (the latter is not *necessarily* faster here).

If using for-loops, then (as before) the pre-defined functions $\texttt{product}$ and $\texttt{get_multidim_slice}$ may be of use to you.

If using vector operations, then $\texttt{np.tile}$ should be helpful.

In [13]:
def check_equality_of_conditionals(dist1, dist2):
    """Checks if p(Q|E_1) = p(Q|E_2), where E_1 \subseteq E_2

    Args:
        dist1 (tuple): (Q, E1, P1) as defined in main text
        dist2 (tuple): (Q, E2, P2) as defined in main text

    Returns
        bool: whether or not the two distributions are equal

    """
    is_valid_dist(dist1)
    is_valid_dist(dist2)
    Q1, E1, P1 = dist1
    Q2, E2, P2 = dist2
    P1 = deepcopy(P1); P2 = deepcopy(P2)  # prevents us from altering the original array
    
    # *** YOUR CODE HERE ***

# QUESTION 3B

This time, we haven't written a test for your $\texttt{check_equality_of_conditionals}$ function. Instead, you're going to test it yourself, by using it to confirm an independence relationship.

We have loaded a distribution and a DAG over which the distribution factorises (represented as a dictionary mapping each variable to its parents).

In [14]:
P = np.load("jpd_8.npz")["P"]
dist = ({i for i in range(P.ndim)}, set(), P)

with open(os.getcwd() + "/graph_8.json", "r") as f:
    graph = json.load(f)
    print(graph)

{'0': [], '1': [0], '2': [1], '3': [1, 2], '4': [0, 2]}


- With pen & paper, draw the graph and verify via d-separation that $x_3 \perp \!\!\! \perp  x_0 | x_1$
- Now verify the same independence numerically with the code you have written so far: $\texttt{marginalise}, \texttt{condition}$, and $ \texttt{check_equality_of_conditionals}$.

In [15]:
# *** YOUR CODE HERE ***

# QUESTION 4A

Write a function $\texttt{does_factorise_over_graph}$ that takes two inputs:
- A joint distribution $(Q, \emptyset, P)$, where $Q=\{0, \ldots, n-1\}$
- A DAG $G$, represented as a dictionary that maps each variable to its parents

and outputs True if the distribution does indeed factorise over $G$, and False otherwise.

You can assume that the ordering $0, \ldots, n-1$ is a valid topological order (you do not need to check this).

You should make use of all three methods you have implemented so far: $\texttt{marginalise}, \texttt{condition}$, and $ \texttt{check_equality_of_conditionals}$.

Do not worry too much about writing efficient code. It just needs to be functional.

In [16]:
def does_factorise_over_graph(dist, G):
    """Checks if dist factorises according to G

    Args:
        dist (tuple): (Q, {}, P) as defined in main text
        G (dict: int --> list of ints): mapping from each variables to its parents

    Returns
        bool: whether or ndist factorises according to G

    """
    is_valid_dist(dist)
    Q, E, P = dist
    P = deepcopy(P)  # prevents us from altering the original array
    
    # *** YOUR CODE HERE ***

# QUESTION 4B

In the cell below, we load 4 distributions and graphs for you. For each distribution, determine which of the graphs it factorises over (there can be multiple).

In [17]:
dists = []
graphs = []
for i in range(4, 8):
    jpd = np.load("jpd_{}.npz".format(i))["P"]
    dist = ({i for i in range(jpd.ndim)}, set(), jpd)
    dists.append(dist)
    
    with open(os.getcwd() + "/graph_{}.json".format(i), "r") as f:
        graph = json.load(f)
        graphs.append(graph)
        print(graph)

{'0': [], '1': [0], '2': [0], '3': [0, 1, 2], '4': [0, 3], '5': [2, 3], '6': [4, 5]}
{'0': [], '1': [], '2': [0, 1], '3': [0, 2], '4': [1, 2], '5': [2, 4], '6': [3]}
{'0': [], '1': [], '2': [0, 1], '3': [0, 2], '4': [0, 1, 2, 3], '5': [1, 2, 3, 4], '6': [3, 4, 5]}
{'0': [], '1': [], '2': [0, 1], '3': [2], '4': [0, 3], '5': [1, 3], '6': [4, 5]}


In [ ]:
# *** YOUR CODE HERE ***